<a href="https://colab.research.google.com/github/thomasbhard/mic-model-retraining/blob/master/mic_model_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retraining the Model with new recordings from the app

- Make sure the json file with the firebase keys is in the same directory as this notebook!

- Upload the h5 model you want to update and set the model filename

- Don't forget to download all the files for later use if you run this in a colab environment.


**Have fun & be safe**

In [0]:
# setup workspace
pip install firebase_admin

     |████████████████████████████████| 92kB 5.5MB/s 
     |████████████████████████████████| 81kB 6.1MB/s 
     |████████████████████████████████| 337kB 11.4MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
ERROR: google-cloud-core 1.2.0 has requirement google-api-core<2.0.0dev,>=1.16.0, but you'll have google-api-core 1.15.0 which is incompatible.
ERROR: google-cloud-bigquery 1.21.0 has requirement google-resumable-media!=0.4.0,<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 0.5.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Found existing installation: google-resumable-media 0.4.1
   

In [0]:
# filenames to load and store models
current_model = 'm1578220494_first=rel_batTrues12.h5'
new_model = 'retrained_model.h5'
new_model_lite = 'retrainded_model.tflite'

In [0]:
# setup firebase
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore


cred = credentials.Certificate('mic-training-firebase-adminsdk-upqap-539c749b5b.json')
firebase_admin.initialize_app(cred)

db = firestore.client()
print('Firebase connection successful!')

Firebase connection successful!


In [0]:
# Collect data
collection = db.collection(u'recordings')

recordings = collection.stream()

features = []
labels = []

for recording in recordings:
  data = recording.to_dict()
  samples = data['samples']
  label = data['label']
  # print(label)

  features.append(samples)
  labels.append(label)

print(str(len(features)) + ' features collected')
print(str(len(labels)) + ' labels collected')


385 features collected
385 labels collected


In [0]:
# Process features and labels
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

# change full instrument names to one_hot_encoded label vectors
instruments = ['cel', 'cla', 'flu', 'gac', 'gel', 'org', 'pia', 'sax', 'tru', 'vio']
fullnames = {'Cello': 'cel','Clarinet': 'cla','Flute': 'flu','Acustic Guitar': 'gac', 'Electric Guitar': 'gel','Organ': 'org', 'Piano': 'pia','Saxophone': 'sax','Trumpet': 'tru','Violin': 'vio'}

label_encoder = LabelEncoder()
label_encoder.fit(instruments)

labels_trans = [fullnames[label] for label in labels]
labels_enc = label_encoder.transform(labels_trans)
labels_one_hot = to_categorical(labels_enc, num_classes=10)

# scale each feature vector to np array between -1.0 and 1.0
features_np = []

for feature in features:
  feature_np = np.array(feature, dtype=np.float32)
  feature_max = np.max(np.abs(feature_np))
  if max != 0:
    feature_np /= feature_max
  features_np.append(feature_np)

print('Processed features and labels')

Using TensorFlow backend.


Processed features and labels


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


In [0]:
# create dataframe
import pandas as pd


df_features = pd.DataFrame(features_np)
df_features.columns = ['Sample ' + str(i) for i in range(2048)]

df_labels = pd.DataFrame(labels_one_hot)
df_labels.columns = instruments

assert len(df_features.index) == len(df_labels.index)

df = pd.concat([df_features, df_labels], axis=1, join='inner')

print(df.head())

In [0]:
# save csv
import time

outfilebase = time.strftime("%Y%m%d-%H%M%S", time.gmtime(time.time()))
outfilename = outfilebase + '-slice-2048.csv'

df.to_csv(outfilename)

In [0]:
# Prepare dataframe for training
df = df.sample(frac=1.0)

X = df.iloc[:, :-10]
X = np.expand_dims(X, axis=2)

Y = df.iloc[:, -10:].values

In [0]:
# update model
from tensorflow.keras.models import load_model

model = load_model(current_model)
model.train_on_batch(X, y=Y)
model.save(new_model)

In [0]:
# convert to tflite
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model_file(new_model)
tflite_model = converter.convert()

file = open(new_model_lite, 'wb' ) 
file.write(tflite_model)